# Final project - 5/3/2025

### Add libarary

In [1]:
import torch
import torch.nn as nn
import numpy as np
import Quantizer
import quantizationItamar

### Task 1:
Helper class to single conv layer
note: we add bias

In [3]:
class QuantizedConvLayer(nn.Module):
    def __init__(self, quantized_weights, original_shape, bias=False):
        super().__init__()
        self.weights = torch.from_numpy(quantized_weights).float().view(original_shape)
        
        if bias:
            self.bias = nn.Parameter(torch.ones(original_shape[0]))  # אתחול הביאס ל-1
        else:
            self.bias = None

    def forward(self, x):
        return nn.functional.conv2d(x, self.weights, bias=self.bias, stride=1, padding=1)


### Task 2:
Single Layer Model

In [12]:
# יצירת מודל עם שכבה אחת
class SingleLayerModel(nn.Module):
    def __init__(self, quantized_weights, original_shape, bias=True):
        # Initialize the object with quantized weights and other parameters
        self.quantized_weights = quantized_weights  # Store quantized_weights in the instance
        self.original_shape = original_shape
        self.bias = bias
    
    def forward(self, x):
        return self.layer(x)


### Task 3:
Randome vector- quantized and dequantized

In [13]:
# יצירת וקטור לדוגמה לבדיקת הכימות
vec = np.random.randn(100).astype(np.float32)  # משקלים אקראיים בגודל 3x3x3, 64 פילטרים

original_shape = (10,10)

# כימות הווקטור
grid = quantizationItamar.generate_grid(8, False)
quantized_weights, scale, z = Quantizer.quantize(vec=vec, grid=grid)

# השוואת וקטורים לפני ואחרי הכימות
dequantized_weights = Quantizer.dequantize(quantized_weights, scale, z)

vector_difference = np.abs(vec - dequantized_weights.flatten()).mean()

# חישוב השגיאה היחסית והמקסימלית
relative_errors = np.abs((vec - dequantized_weights.flatten()) / vec)
relative_errors = np.where(np.isfinite(relative_errors), relative_errors, 0)  # טיפול במקרים של חלוקה באפס

average_relative_error = np.mean(relative_errors)
max_relative_error = np.max(relative_errors)

print("Average relative error:", average_relative_error)
print("Maximum relative error:", max_relative_error)




Average relative error: 0.04171245743774483
Maximum relative error: 1.0


### write the results into file

In [6]:
with open("vectors_output.txt", "w") as f:
    f.write("Original vector:\n")
    np.savetxt(f, vec.reshape(-100, 100), fmt="%.6f")
    f.write("\nDequantized vector:\n")
    np.savetxt(f, dequantized_weights.reshape(-100, 100), fmt="%.6f")
    f.write("\nvector_difference:\n")
    np.savetxt(f, vector_difference.reshape(-1, 1), fmt="%.6f")
print("Vectors saved to vectors_output.txt")


Vectors saved to vectors_output.txt


### Output from file:

Original vector:
-0.111487 0.850883 -1.562684 -0.561151 0.165896 -0.616522 -0.294702 -0.178508 -0.119455 0.856403 -0.005942 -0.944988 -0.185713 -1.316645 0.863703 0.146863 2.823082 0.450654 -1.708542 -0.476596 1.867064 0.389970 -1.967263 -0.890203 0.928423 0.202063 0.858497 -0.375497 0.413610 -0.777870 -0.032337 1.293657 0.966133 -0.319092 -1.100120 0.950536 -0.845570 1.726389 1.885617 1.256837 1.217571 -0.203107 1.917088 0.126608 -0.931344 -1.840507 -1.579605 -0.550749 -0.427611 -0.631418 2.302883 1.569178 0.134905 -0.141346 -1.992737 0.787142 0.997367 1.543908 1.267690 -0.805269 0.919002 0.291717 0.560167 -0.341881 -1.454660 -1.803350 1.644349 1.351046 -0.138112 -1.322578 0.343876 -0.676363 -0.518376 0.261224 1.250156 1.880378 -0.944845 -0.257316 -0.693713 0.533303 0.203999 -0.596182 0.611321 -0.632613 0.697375 -0.829484 0.523490 0.563056 -0.220699 -0.058949 0.108626 0.239777 -0.356313 0.832974 0.551994 0.847768 -2.287619 -0.387235 0.229343 1.890911

Dequantized vector:
-0.100210 0.841762 -1.543231 -0.541133 0.160336 -0.601259 -0.280587 -0.160336 -0.100210 0.841762 0.000000 -0.941972 -0.180378 -1.302728 0.861804 0.140294 2.805875 0.440923 -1.703567 -0.460965 1.863903 0.380797 -1.964112 -0.881846 0.921930 0.200420 0.841762 -0.360755 0.400839 -0.761595 -0.020042 1.282686 0.962014 -0.300629 -1.082266 0.941972 -0.841762 1.723609 1.883945 1.242602 1.202518 -0.200420 1.903987 0.120252 -0.921930 -1.823819 -1.563273 -0.541133 -0.420881 -0.621301 2.284784 1.563273 0.120252 -0.140294 -1.984154 0.781637 0.982056 1.543231 1.262644 -0.801679 0.901888 0.280587 0.541133 -0.340713 -1.443021 -1.783735 1.643441 1.342812 -0.120252 -1.302728 0.340713 -0.661385 -0.501049 0.260546 1.242602 1.863903 -0.941972 -0.240504 -0.681427 0.521091 0.200420 -0.581217 0.601259 -0.621301 0.681427 -0.821720 0.521091 0.561175 -0.220462 -0.040084 0.100210 0.220462 -0.340713 0.821720 0.541133 0.841762 -2.284784 -0.380797 0.220462 1.883945

vector_difference:
0.010174


### Task 4:
One layer network and quntize

In [16]:
# do quantized
quantized_weights, scale, z = Quantizer.quantize(vec=vec, grid=grid)

# Initialize model with quantized weights
model = SingleLayerModel(quantized_weights, original_shape, bias=True)

# Dequantizing weights
dequantized_weights = Quantizer.dequantize(model.quantized_weights , scale, z)

# Calculate vector difference mean
vector_difference = np.abs(vec - dequantized_weights.flatten()).mean()

# Calculate relative 
errors = np.abs((vec - dequantized_weights.flatten()) / vec)

# Handling NaNs or infinities in relative_errors (for division by zero)
relative_errors = np.where(np.isfinite(relative_errors), relative_errors, 0)

# Compute average and maximum errors
average_error = np.mean(relative_errors)
max_error = np.max(relative_errors)

# Print the results
print("Vector difference (mean absolute error):", vector_difference)
print("Average relative error:", average_error)
print("Maximum relative error:", max_error)

Vector difference (mean absolute error): 0.009622683358725714
Average relative error: 0.04171245743774483
Maximum relative error: 1.0


### Task 5: genrate vectors with diffrente range etc 
### Examine the results of each type of vector
function 1: create_vectors

In [17]:
def create_vectors():
    # Randome vector
    vec_random = np.random.randn(100).astype(np.float32)
    
    # pos velue
    vec_positive = np.abs(np.random.randn(100).astype(np.float32))
    
    #vector with valus in range [-10,10]
    vec_range = np.random.uniform(-10, 10, 100).astype(np.float32)
    
    # וקטור עם ערכים מאוד קטנים (כדי לבדוק את דיוק הקוונטיזציה)
    vec_small = np.random.randn(100).astype(np.float32) * 1e-6
    
    return vec_random, vec_positive, vec_range, vec_small


In [27]:
# Open a file to write the results to
with open('quantization_results.txt', 'w') as file:
    
    # For each vector, perform quantization, dequantization and evaluation

    for vec in vectors:
        # Write the vector type to the file
        file.write(f"Processing vector with values:\n{vec}...\n")  # Print part of the vector
        
        # Generate the grid for quantization
        grid = quantizationItamar.generate_grid(8, True)

        # Perform quantization
        quantized_weights, scale, z = Quantizer.quantize(vec=vec, grid=grid)

        # Create model with quantized weights
        model = SingleLayerModel(quantized_weights, original_shape=(10, 10), bias=True)

        # Perform dequantization
        dequantized_weights = Quantizer.dequantize(model.quantized_weights, scale, z)

        # Write results to the file
        file.write(f"dequantized_weights: {dequantized_weights}\n")

        # Write intermediate results to the file (vector difference, etc.)
        vector_difference = np.abs(vec - dequantized_weights.flatten()).mean()

        # Calculate relative errors
        relative_errors = np.abs((vec - dequantized_weights.flatten()) / vec)
        relative_errors = np.where(np.isfinite(relative_errors), relative_errors, 0)  # Handle division by zero

        # Calculate average and maximum relative errors
        average_relative_error = np.mean(relative_errors)
        max_relative_error = np.max(relative_errors)

        # Write results to the file
        file.write(f"Vector difference (mean absolute error): {vector_difference}\n")
        file.write(f"Average relative error: {average_relative_error}\n")
        file.write(f"Maximum relative error: {max_relative_error}\n\n")

       

### Output 1:
###  For a Randome vector:
Processing vector with values:
[ 1.845 -0.024  0.657 -1.122  0.105 -0.563  0.009 -1.249  1.207 -0.408
  1.842  1.724  1.019  0.537 -1.06  -0.054 -1.125 -0.246 -1.27   0.953
 -0.697 -0.809 -0.855 -1.088 -1.315 -0.516  0.362 -0.311 -1.889  1.169
 -0.067 -0.749  1.73   0.041 -0.83   0.223  1.78   0.438  0.686 -0.163
 -0.727 -1.002  0.992  2.056 -1.237  1.357  1.3   -0.996  0.375 -0.469
  0.912 -0.283  0.868  1.727  0.022 -0.386  1.109  0.573  0.217 -0.621
  0.828 -2.017  0.235  1.48  -1.314 -0.152  0.035  0.403  1.     0.301
 -0.225  1.083 -0.268  0.379 -0.723  0.899  2.327  0.471 -0.244  2.279
 -0.26  -0.105 -2.934  1.683 -1.04   1.013  1.58  -1.523  0.919 -1.786
  0.55   1.073  0.158 -1.12  -0.606 -0.656 -0.51  -1.438 -0.295  1.27 ]


dequantized_weights: [ 1.836 -0.021  0.64  -1.114  0.103 -0.557  0.    -1.238  1.197 -0.392
  1.836  1.713  1.011  0.536 -1.052 -0.041 -1.114 -0.227 -1.259  0.949
 -0.681 -0.805 -0.846 -1.073 -1.3   -0.516  0.351 -0.31  -1.878  1.156
 -0.062 -0.743  1.713  0.021 -0.825  0.206  1.775  0.433  0.681 -0.144
 -0.722 -0.99   0.99   2.043 -1.217  1.341  1.3   -0.99   0.371 -0.454
  0.908 -0.268  0.867  1.713  0.021 -0.371  1.094  0.557  0.206 -0.619
  0.825 -2.002  0.227  1.465 -1.3   -0.144  0.021  0.392  0.99   0.289
 -0.206  1.073 -0.248  0.371 -0.722  0.887  2.311  0.454 -0.227  2.27
 -0.248 -0.103 -2.93   1.671 -1.032  1.011  1.568 -1.506  0.908 -1.775
  0.536  1.073  0.144 -1.114 -0.598 -0.64  -0.495 -1.424 -0.289  1.259]


Vector difference (mean absolute error): 0.009988984240985902

Average relative error: 0.04160369772987067

Maximum relative error: 1.0

### Output 2:
###  For a positve vector:
Processing vector with values:
[0.842 0.564 0.53  0.306 0.552 1.177 0.127 0.373 0.325 0.024 1.546 1.958
 0.467 0.3   0.05  0.922 0.715 0.041 2.099 1.996 0.097 0.81  1.928 0.05
 1.063 0.562 0.257 0.074 1.155 1.052 0.429 1.142 0.131 0.013 0.644 0.715
 2.019 0.478 0.384 0.812 0.365 0.229 0.724 0.124 0.753 0.4   0.176 0.206
 0.662 0.171 1.264 1.29  0.197 0.169 0.184 0.874 1.28  0.812 0.64  0.115
 0.789 0.249 0.041 0.124 0.527 0.994 0.189 1.081 0.68  0.09  1.544 0.187
 0.613 1.349 0.851 0.272 0.454 2.093 0.354 0.769 2.432 0.326 0.598 0.465
 0.008 0.992 0.449 1.031 0.368 0.304 0.68  0.604 1.092 0.097 0.192 0.385
 0.937 0.405 0.431 0.654]


dequantized_weights: [0.837 0.561 0.523 0.304 0.551 1.169 0.124 0.371 0.323 0.019 1.54  1.949
 0.466 0.295 0.048 0.922 0.713 0.038 2.092 1.987 0.095 0.808 1.921 0.048
 1.055 0.561 0.257 0.067 1.15  1.046 0.428 1.141 0.124 0.01  0.637 0.713
 2.016 0.475 0.38  0.808 0.361 0.228 0.723 0.124 0.751 0.399 0.171 0.2
 0.656 0.162 1.255 1.284 0.19  0.162 0.181 0.865 1.274 0.808 0.637 0.114
 0.78  0.247 0.038 0.124 0.523 0.989 0.181 1.074 0.675 0.086 1.54  0.181
 0.608 1.341 0.846 0.266 0.447 2.092 0.352 0.761 2.424 0.323 0.589 0.456
 0.    0.989 0.447 1.027 0.361 0.304 0.675 0.599 1.084 0.095 0.19  0.38
 0.932 0.399 0.428 0.647]


Vector difference (mean absolute error): 0.00449026730327917


Average relative error: 0.027712205461241517


Maximum relative error: 1.0

### Output 3:
###  For a vector with number in rang [-10,10]:

Processing vector with values:
[-7.486  6.453  3.698  1.974 -8.849 -0.654  0.024  3.682 -7.765 -7.426
 -4.756  3.291 -0.414 -4.156 -0.233 -1.8   -0.76  -3.587  8.459  0.476
  1.153 -7.993  0.367  6.308 -8.249  7.706  7.149 -4.739 -7.791 -8.182
 -8.857 -1.62   5.834 -5.853  9.697 -9.905  1.939 -9.207  7.317 -4.41
  8.908 -0.757  7.405 -1.313  9.301  5.434  2.446  1.684 -0.564 -8.49
 -9.02   5.381  5.913  3.296  3.75  -5.363 -7.126  5.366  4.749  0.752
 -7.923  6.479 -6.492  4.873 -0.831 -5.933 -1.023 -6.624  7.518  8.685
  6.823  7.717 -2.144 -7.221 -1.954 -2.586  5.122  8.938 -7.526  6.726
 -6.834  9.901 -4.333 -5.694 -1.824  5.601 -8.753  7.033 -7.669  3.91
 -5.535  6.585  7.296  8.445  4.108  1.842 -1.195 -8.319 -2.418  3.343]


dequantized_weights: [-7.456  6.447  3.651  1.942 -8.777 -0.621  0.     3.651 -7.689 -7.379
 -4.738  3.262 -0.388 -4.117 -0.233 -1.786 -0.699 -3.573  8.388  0.466
  1.087 -7.922  0.311  6.291 -8.233  7.689  7.146 -4.738 -7.767 -8.155
 -8.854 -1.553  5.825 -5.825  9.631 -9.864  1.864 -9.165  7.301 -4.35
  8.854 -0.699  7.379 -1.243  9.243  5.359  2.408  1.631 -0.544 -8.466
 -9.01   5.359  5.903  3.262  3.728 -5.359 -7.068  5.359  4.738  0.699
 -7.922  6.447 -6.447  4.816 -0.777 -5.903 -1.01  -6.602  7.456  8.621
  6.757  7.689 -2.097 -7.146 -1.942 -2.563  5.049  8.932 -7.456  6.68
 -6.757  9.864 -4.272 -5.67  -1.786  5.592 -8.699  6.99  -7.612  3.884
 -5.515  6.524  7.223  8.388  4.039  1.786 -1.165 -8.311 -2.408  3.34 ]


Vector difference (mean absolute error): 0.03731790080447429


Average relative error: 0.024350610271637874


Maximum relative error: 1.0

### Output 4:
###  For a small number vector:

Processing vector with values:
[-2.255e-07 -1.138e-06 -1.651e-06  3.693e-07  1.656e-06  1.146e-06
  7.117e-07 -2.367e-07  9.144e-08 -3.804e-07 -9.244e-08 -3.330e-07
  2.401e-07  8.202e-07 -7.476e-07  5.232e-07  1.132e-06 -1.039e-06
 -1.128e-07  5.556e-07 -2.252e-07 -1.458e-06 -1.388e-06  7.225e-07
  5.900e-07 -1.102e-06 -5.305e-07  1.218e-07 -1.339e-06  5.616e-08
 -1.877e-06 -1.625e-07 -7.366e-08  1.911e-07 -5.086e-07 -1.099e-06
 -3.371e-07 -3.181e-07 -1.950e-06  1.368e-06  1.594e-07  2.152e-08
  1.294e-06 -3.561e-07  1.483e-06  4.243e-07 -5.544e-07  1.357e-06
  1.551e-06  2.650e-06 -9.056e-07  7.431e-07 -8.916e-07 -1.031e-06
  1.240e-06 -8.245e-07 -6.069e-07 -5.353e-07  1.293e-06 -5.546e-07
  3.173e-07  4.125e-07 -1.560e-06  7.750e-08 -2.279e-06  2.132e-06
 -1.090e-06  3.612e-07 -9.459e-07  2.980e-07 -7.555e-07  8.842e-07
  6.081e-07  6.104e-07  4.233e-07 -3.547e-07 -1.242e-06 -1.045e-06
 -5.698e-07  1.037e-06 -1.407e-07 -4.682e-07  4.941e-07 -2.270e-07
 -3.758e-07  5.382e-07 -1.437e-06 -1.101e-06  3.997e-07  2.200e-06
  4.046e-08 -4.211e-07  1.344e-06  4.561e-08 -1.512e-06 -2.675e-07
  1.218e-07 -3.649e-07  4.414e-07  4.752e-07]


dequantized_weights: [-2.126e-07 -1.121e-06 -1.643e-06  3.672e-07  1.643e-06  1.140e-06
  6.958e-07 -2.319e-07  7.731e-08 -3.672e-07 -7.731e-08 -3.286e-07
  2.319e-07  8.118e-07 -7.345e-07  5.219e-07  1.121e-06 -1.024e-06
 -9.664e-08  5.412e-07 -2.126e-07 -1.450e-06 -1.372e-06  7.152e-07
  5.799e-07 -1.102e-06 -5.219e-07  1.160e-07 -1.334e-06  3.866e-08
 -1.875e-06 -1.546e-07 -5.799e-08  1.740e-07 -5.025e-07 -1.082e-06
 -3.286e-07 -3.093e-07 -1.933e-06  1.353e-06  1.546e-07  1.933e-08
  1.276e-06 -3.479e-07  1.469e-06  4.059e-07 -5.412e-07  1.353e-06
  1.546e-06  2.648e-06 -8.891e-07  7.345e-07 -8.891e-07 -1.024e-06
  1.237e-06 -8.118e-07 -5.992e-07 -5.219e-07  1.276e-06 -5.412e-07
  3.093e-07  4.059e-07 -1.546e-06  7.731e-08 -2.261e-06  2.126e-06
 -1.082e-06  3.479e-07 -9.278e-07  2.899e-07 -7.538e-07  8.698e-07
  5.992e-07  5.992e-07  4.059e-07 -3.479e-07 -1.237e-06 -1.044e-06
 -5.605e-07  1.024e-06 -1.353e-07 -4.639e-07  4.832e-07 -2.126e-07
 -3.672e-07  5.219e-07 -1.430e-06 -1.082e-06  3.866e-07  2.184e-06
  3.866e-08 -4.059e-07  1.334e-06  3.866e-08 -1.508e-06 -2.513e-07
  1.160e-07 -3.479e-07  4.252e-07  4.639e-07]

Vector difference (mean absolute error): 1.0179261771897355e-08

Average relative error: 0.030824232498440164

Maximum relative error: 0.311715113852703



### Task 6:
### Compare between INT8 and INT16 QUANTIZE

In [33]:
import numpy as np

# Generate test vectors
vectors = [
    np.random.randn(1000).astype(np.float32),  # Normal distribution
    np.random.uniform(-1, 1, 1000).astype(np.float32),  # Uniform distribution
    np.linspace(-10, 10, 1000).astype(np.float32)  # Linear ramp
]

# Open a file for saving results
with open('quantization_16bit_vs_8bit.txt', 'w') as file:
    for vec in vectors:
        file.write(f"Processing vector with first values: {vec[:5]}...\n")

        # Generate grids for both 16-bit and 8-bit
        grid_16bit = quantizationItamar.generate_grid(16, False)
        grid_8bit = quantizationItamar.generate_grid(8, False)

        try:
            # Perform 16-bit quantization
            quantized_16bit, scale_16bit, z_16bit = Quantizer.quantize(vec, grid_16bit)
            dequantized_16bit = Quantizer.dequantize(quantized_16bit, scale_16bit, z_16bit)

            # Perform 8-bit quantization
            quantized_8bit, scale_8bit, z_8bit = Quantizer.quantize(vec, grid_8bit)
            dequantized_8bit = Quantizer.dequantize(quantized_8bit, scale_8bit, z_8bit)

            # Compute errors
            error_16bit = np.abs(vec - dequantized_16bit).mean()
            max_error_16bit = np.abs(vec - dequantized_16bit).max()

            error_8bit = np.abs(vec - dequantized_8bit).mean()
            max_error_8bit = np.abs(vec - dequantized_8bit).max()

            # Write results
            file.write(f"16-bit Quantization -> Mean Error: {error_16bit}, Max Error: {max_error_16bit}\n")
            file.write(f"8-bit Quantization  -> Mean Error: {error_8bit}, Max Error: {max_error_8bit}\n\n")

        except Exception as e:
            file.write(f"Error processing vector: {str(e)}\n\n")

print("Quantization analysis completed. Results saved to 'quantization_16bit_vs_8bit.txt'.")


Quantization analysis completed. Results saved to 'quantization_16bit_vs_8bit.txt'.


### The results of this conpare:
### First vector:
Processing vector with first values: [ 0.255  0.02   1.868 -0.964  0.563]...
16-bit Quantization : 

Mean Error: 5.056971670431684e-05

Max Error: 0.00010206876043378221

8-bit Quantization:

 Mean Error: 0.013173053463762753 

 Max Error: 0.026206773870131705

### second vector:
Processing vector with first values: [-0.813 -0.25  -0.583  0.722 -0.494]...

16-bit Quantization:

Mean Error: 1.5144269051737838e-05

Max Error: 3.0485269083280198e-05

8-bit Quantization:

Mean Error: 0.003858430505117119

Max Error: 0.007832529147466072

### thired vector
Processing vector with first values: [-10.    -9.98  -9.96  -9.94  -9.92]...

16-bit Quantization:

Mean Error: 0.0001477456901089138

Max Error: 0.00030483151406279774

8-bit Quantization: 

Mean Error: 0.039514801461834

Max Error: 0.07831359552402123

